In [ ]:
import pandas as pd

df = pd.read_csv("../src/state/exp.csv", index_col=0).reset_index(drop=True)
df.head()

In [ ]:
Lat_max, Lat_min = df.stopLat.max(), df.stopLat.min()
Lon_max, Lon_min = df.stopLon.max(), df.stopLon.min()

In [ ]:
map_3city_points = df[['stopLon', 'stopLat']].values

In [ ]:
import numpy as np
from scipy.spatial import Voronoi

stops = df.coord.values
ids = np.array(map_3city_points)
vor = Voronoi(ids)
polygons = {}
for id, region_index in enumerate(vor.point_region):
    points = []
    for vertex_index in vor.regions[region_index]:
        if vertex_index != -1:  # the library uses this for infinity
            points.append(list(vor.vertices[vertex_index]))
    points.append(points[0])
    polygons[id]=points

In [ ]:
stops_indexes = [row['coord'] for index, row in df.iterrows()]

In [ ]:
data_geoJson = [{index: coordinates} for index, coordinates in zip(stops_indexes, polygons.values())]

In [ ]:
from shapely.geometry import Point, LineString, Polygon

In [ ]:
boundary = np.array([[18.948932, 54.354509],
                     [18.606983, 54.436059],
                     [18.558918, 54.596288],
                     [18.341938, 54.594697],
                     [18.411136, 54.388688],
                     [18.539508, 54.264611],
                     [18.929380, 54.297227]])

In [ ]:
boundary_3city = Polygon(boundary).buffer(0.00001)
data_geoJson_boundary = []
empty_test = []
for area in data_geoJson:
    for k, v in area.items():
        poly = Polygon(list(v)).buffer(0.00001)
        new_poly = boundary_3city.intersection(poly)
        empty_test.append(new_poly.is_empty)
        if not new_poly.is_empty:
            data_geoJson_boundary.append({k:[[x, y] for x, y in zip(new_poly.exterior.coords.xy[0],
                                                                    new_poly.exterior.coords.xy[1])]})
        else:
            data_geoJson_boundary.append(area)
len(data_geoJson_boundary)

In [ ]:
set(empty_test)

In [ ]:
data_geoJson[260]

In [ ]:
cell = Polygon(list(data_geoJson[260].values())[0]).buffer(0.00001)

new_polygon = cell.intersection(boundary_3city)

In [ ]:
cell

In [ ]:
boundary_3city

In [ ]:
new_polygon

In [1]:
import logging

In [ ]:
geoJson_3city = {"type":"FeatureCollection","features":[
                    {"type":"Feature",
                     "id":list(zone.keys())[0],
                     "properties":{"name":"3city"},
                     "geometry":{"type":"Polygon",
                                 "coordinates": list(zone.values())}} for zone in data_geoJson_boundary]}

In [ ]:
import json
with open('../src/state/geoJson_3city_boundary.json', 'w') as fp:
    json.dump(geoJson_3city, fp)

In [ ]:
import folium
import os

city_geo = os.path.join('../src/state/', 'geoJson_3city_boundary.json')
# city_geo = os.path.join('../src/state/', 'geoJson_3city.json')

time_data = pd.read_csv("../src/state/exp.csv", index_col=0).reset_index(drop=True).\
                                                             rename(columns={'coord':'id'})
time_data['time_min'] = time_data['time_sec'] / 60

bins = [0, 1, 2, 3, 4, 5, 6, 12]

m = folium.Map(location=[54.441564, 18.562754], zoom_start = 10)

folium.Choropleth(
    geo_data=city_geo,
    name='choropleth',
    data=time_data,
    columns=['id', 'time_min'],
    key_on='feature.id',
    line_weight=2,
    fill_color='YlOrRd',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name='Time (min)',
    bins=9,
    reset=True
).add_to(m)

df['Lat'] = df['stopLat'].astype(float)
df['Lon'] = df['stopLon'].astype(float)

heat_df = df[['Lat', 'Lon']]
heat_data = [[row['Lat'], row['Lon']] for index, row in heat_df.iterrows()]

for index, row in heat_df.iterrows():
    folium.Circle(location=[row['Lat'], row['Lon']],
                  radius=10,
                  color=None,
                  fill=True,
                  fill_color="black",
                  fill_opacity=1).add_to(m)

m#.save("folium_choropleth_3city.html")